In [3]:
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import pytz
from dotenv import load_dotenv
import os

In [10]:
# Carrega o .env dentro da pasta sandbox
load_dotenv(".env")

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_API_KEY = os.getenv("SUPABASE_API_KEY")
SUPABASE_TABLE = 'consumo_agua'

if not SUPABASE_URL or not SUPABASE_API_KEY:
    raise ValueError("⚠️ SUPABASE_URL ou SUPABASE_API_KEY não carregadas corretamente!")

In [11]:
HEADERS = {
    "apikey": SUPABASE_API_KEY,
    "Authorization": f"Bearer {SUPABASE_API_KEY}",
    "Content-Type": "application/json",
}

In [12]:
# 🕒 Definir fuso horário UTC-3 (Brasil)
UTC_MINUS_3 = pytz.timezone("America/Sao_Paulo")

In [13]:
# 📌 Função para obter histórico de consumo
def obter_historico():
    response = requests.get(f"{SUPABASE_URL}/rest/v1/{SUPABASE_TABLE}?select=*", headers=HEADERS)
    if response.status_code == 200:
        df = pd.DataFrame(response.json())
        if not df.empty:
            df["data_hora"] = pd.to_datetime(df["data_hora"]).dt.tz_convert(UTC_MINUS_3)
            df["data"] = df["data_hora"].dt.date
            df["hora"] = df["data_hora"].dt.hour
        return df
    return pd.DataFrame()

In [14]:
# 🔄 Carregar os dados
historico = obter_historico()
if historico.empty:
    print("Nenhum dado encontrado.")
else:
    print(f"{len(historico)} registros carregados.")

35 registros carregados.


In [15]:
# 🎯 Meta diária de consumo (ml)
META_DIARIA = 3000

In [16]:
# 📊 Gráfico: Consumo Total por Dia
historico_usuario = historico.groupby("data")["quantidade_ml"].sum().reset_index()
fig_dia = px.bar(historico_usuario, x="data", y="quantidade_ml", title="Consumo Total por Dia", text_auto=True)
fig_dia.show()

In [17]:
# 📈 Gráfico: Evolução do Consumo Diário (%) da Meta
historico_usuario["percentual_meta"] = (historico_usuario["quantidade_ml"] / META_DIARIA) * 100
fig_meta = px.line(
    historico_usuario,
    x="data",
    y="percentual_meta",
    markers=True,
    title="Evolução do Consumo Diário (%) da Meta",
    text=historico_usuario["percentual_meta"].round(1).astype(str) + "%"
)
fig_meta.add_trace(go.Scatter(
    x=historico_usuario["data"],
    y=[100] * len(historico_usuario),
    mode="lines",
    line=dict(dash="dash", color="red"),
    name="Meta 100%"
))
fig_meta.show()

In [18]:
# ⏳ Gráfico: Consumo Médio por Hora
consumo_por_hora = historico.groupby("hora")["quantidade_ml"].mean().reset_index()
fig_hora = px.bar(
    consumo_por_hora,
    x="hora",
    y="quantidade_ml",
    title="Consumo Médio por Hora",
    labels={"hora": "Hora do Dia", "quantidade_ml": "Consumo Médio (ml)"},
    text_auto=True
)
fig_hora.show()